El objetivo es encotrar la informacion mas posible, y que sea de calidad. De acuerdo a los criterios de PLD
La tabla de clientes es

| Información requerida                   | Información obtenida                             |
|-----------------------------------------|--------------------------------------------------|
| negocio                                 |                            |                            
| nombre cliente o razón social           |  BCO_NOMBRE                |                            
| número cliente                          |        BCO_CLAVE        |                            
| estatus                                 |         BCO_ESTATUS                   |                            
| número contrato                         |                            |                            
| género                                  |            N/A                |                            
| fecha_nacimiento o de constitución      |                            |                            
| entidad federativa nacimiento           |                            |                            
| país de nacimiento                      |                            |                            
| nacionalidad                            |                            |                            
| profesión                               |                            |                            
| calle                                   |  SUC_DIRECCION     |                            
| número_exterior                         |                            |                           
| número_interior                         |                            |                            
| colonia_urbanización                    |                            |                            
| delegación_municipio                    |                            |         
| ciudad_población                        |   SUC_CDS_CLAVE    CDS_NOMBRE         |                              
| entidad_federativa                      |    SUC_EDO_CLAVE         |
| código postal                           |                            |                              
| país                                    |   SUC_PAI_CLAVE                 |                      
| teléfono                                | SIPE.SIPE_CONTACTOS SC ON BAD.SUC_NUMERO= SC.CTO_SUC_NUMERO |     
| correo_electrónico                      |                            |        
| rfc                                     |    SUC_RFC                        |  
| curp                                    |       N/A                     |                                   
| fiel                                    |                            |                                   
| representante legal                     |                            |
| tipo de persona                         |       PM                     |
| clasificación por grado de riesgo       |                            |                            
| pep (persona políticamente expuesta)    |                            |                            
| razón                                   |                            |                             
| producto contratado                     |                            |
| cambios en perfil transaccional         |                            |

In [1]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
from rapidfuzz import process, fuzz
import numpy as np


In [2]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/Clientes_activos_SIPE.dat")

In [3]:
print(df["NOMBRE_O_RAZON_SOCIAL"].value_counts().to_string(max_rows=None))

NOMBRE_O_RAZON_SOCIAL
STANDARD CHARTERED BANK                2
BANK OF NEW YORK MELLON                2
SOCIETE GENERALE                       2
BANCO J.P. MORGAN MEXICO               1
BARCLAYS BANK  PLC NEW YORK            1
BOZANO SIMIOSEN BANK                   1
HSBC MEXICO, S.A.                      1
HSBC BANK USA                          1
COMMERZBANK, FRANKFURT                 1
SOCIEDAD HIPOTECARIA FEDERAL SNC       1
BANK OF AMERICA, N.A.                  1
FIFTH THIRD BANK                       1
CONFIA                                 1
JPMORGAN CHASE NEW YORK                1
BANK OF TOKYO MITSUBISHI MEXICO        1
INTERBANCO, S.A.                       1
KREDIET BANK CORP. NEW  YORK           1
BANCO INTERACCIONES, SA FONDOS BID     1
CREDIT AGRICOLE SA, PARIS              1
ITALBANK INTERNATIONAL PUERTO RICO     1
BANCO INVEX, S.A. INST, BCA. MULT.     1
BARCLAYS BANK LONDON                   1
BANK OF TOKYO MITSUBISHI NEW YORK      1
WELLS FARGO (FORMELY WACHOVIA ) NY 

In [4]:
df["RFC"] = df["RFC"].apply(lambda x: x if pd.notna(x) else "XXXXXXXXXXXXX")
df["RFC"] = df["RFC"].apply(lambda x:  normalizar_cadena(x) if verificar_rfc(normalizar_cadena(x)) else "XXXXXXXXXXXXX")

In [5]:
df['FECHA_DE_ACTUALIZACION'] = datetime.now().strftime('%d/%m/%y')

In [6]:
col_final = ['NEGOCIO', 'NOMBRE_O_RAZON_SOCIAL', 'NUMERO_CLIENTE', 'ESTATUS',
       'NUMERO_CONTRATO', 'GENERO', 'FECHA_NAC_O_CONST',
       'ENTIDAD_FEDERATIVA_NACIMIENTO', 'PAIS_DE_NACIMIENTO', 'NACIONALIDAD',
       'PROFESION', 'CALLE', 'NUMERO_EXTERIOR', 'NUMERO_INTERIOR',
       'COLONIA_URBANIZACION', 'DELEGACION_MUNICIPIO', 'CIUDAD_POBLACION',
       'ENTIDAD_FEDERATIVA', 'CODIGO_POSTAL', 'PAIS', 'TELEFONO',
       'CORREO_ELECTRONICO', 'RFC', 'CURP', 'FIEL', 'REPRESENTANTE_LEGAL',
       'TIPO_PERSONA', 'PRODUCTO_CONTRATADO',
       'SISTEMA_ORIGEN', 'FECHA_DE_ACTUALIZACION']
df = df[col_final]

In [7]:
clt_repetidos = compara_nombres(df, df, "NOMBRE_O_RAZON_SOCIAL", "NOMBRE_O_RAZON_SOCIAL","NUMERO_CLIENTE","NUMERO_CLIENTE", 95)
df_resul, drops_ids = agregar_asociaciones_clientes(clt_repetidos,df)
df = df.drop(drops_ids)
df = pd.concat([df, df_resul], ignore_index=True)
df = df.sort_values("NOMBRE_O_RAZON_SOCIAL")

/home/ale1726/proyects/datalake/clientes/clientes_eda/utils/clt_repetidos.py:182: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_resul = df_resul.replace("-", np.nan)


In [8]:
name_archivo = os.path.join("/home/ale1726/proyects/datalake/clientes/data/ETL","CLientesT_SIPE.csv")
df.to_csv(name_archivo,index=False)
